In [1]:
from splinter import Browser
from selenium.common.exceptions import InvalidElementStateException
from selenium.common.exceptions import ElementNotInteractableException
from bs4 import BeautifulSoup
import pandas as pd
import html5lib
import lxml
import shutil
import time
import urllib3
import requests

In [2]:
# Set the zip codes
codes = [i for i in range(27006, 28909)]
# codes.append('27606')   # just hard code one in
code = codes[0]    # set the variable for zip_and_click
# len(codes)

In [3]:
# inititial website to visit
url = 'https://weather.com'

In [4]:
def chrome():
    '''
    Finds the chromedriver in the system and creates a Chrome browser
    '''
    executable_path = {'executable_path': shutil.which('chromedriver')}
    browser = Browser('chrome', **executable_path)
    return(browser)

In [5]:
def zip_and_click(code):
    '''
    Enter zip codes into the search bar on weather.com and click the first result.
    No return, just leaves browser at the first data page.
    '''
    import time
    
    filled = False
    clicked = False
    
    inputs = browser.find_by_tag('input') # get the search box reference
    search_box = inputs[0]
    while not filled:
        try:
            search_box.fill(code)
            filled = True
        except InvalidElementStateException:
            time.sleep(1)
    while not clicked:
        try:
            browser.click_link_by_partial_href('/weather/today/l')
            print('no exception this time')
            clicked = True
        except ElementNotInteractableException:
            print('exception found....waiting...')
            time.sleep(1)
    return(browser)     

In [6]:
def scrape_now(browser):
    ''' Get observed weather data from the current weather page '''
    divs = browser.find_by_tag('div').first
    nowcard = divs.find_by_tag('section.today_nowcard-container').value
    nowlist = nowcard.split('\n')
    return(nowlist)

In [7]:
def wind_dict(wind_str):
    ''' handle the cases where the wind value is simply "calm" rather than direction and speed '''
    if wind_str == 'Calm':
        return {
            'speed': 0,
            'direction': None
        }
    else:
        return {
            'speed': wind_str.split()[1],
            'direction': wind_str.split()[0]
        }

In [17]:
def scrape_next36(browser):
    ''' Get the "at a glance" data from the next 36 hours '''

    next_36 = {}
    
    # Pulling data from the "next 36 hours" overview displays. There are 5 sections
    # that are pulled from. Each section gets a different label.
    hour_code = {'0':'now',
                '1':'at_start',
                '2':'at_12',
                '3':'at_24',
                '4':'at_36'
                }
    for i in range(5):
        print(f'looking for {hour_code[str(i)]} snapshot')
        try:
            browser.find_by_tag(f'div#daypart-{i}').click()
        except ElementNotInteractableException:
            print('ElementNotInteractableException occured: breaking loop.')
            break
        snapshot = browser.find_by_tag(f'div#daypart-{i}')
        snap_values = snapshot.value
        details = browser.find_by_tag('span.wx-detail-value')
        detaillist = [details[j].value for j in range(4)]
        print(f'completed section {i} snapshot')

        #store the data
        next_36.update({hour_code[str(i)]: {'hour': 'CALCULATED HOUR OF PREDICTION',
                                            'condition': snap_values.split('\n')[1],
                                            'high/low': snap_values.split('\n')[2],
                                            'temp_f': snap_values.split('\n')[3],
                                            'chance_precip': snap_values.split('\n')[4],
                                            'description': browser.find_by_id(f'dp{i}-details-narrative').first.value,
                                            'wind': wind_dict(detaillist[0]),
#                                             {'speed_mph': detaillist[0].split(' ')[1],
#                                                            'direction': detaillist[0].split(' ')[0]
#                                                    },
                                            'humidity': detaillist[1],
                                            'uv_index': detaillist[2],
                                            'sun': {'rise': detaillist[3],
                                                    'set': detaillist[3]
                                                   }
                                              }
                      })
    return(next_36)

In [9]:
def goto_hourly(browser):
    ''' Take the browser to the hourly numbers '''
    browser.find_by_text('Hourly').click()
    time.sleep(1)
    n=0
    while browser.is_text_present('Next 8 Hours', wait_time=.5):
        n+=1
        print(n)
        next8 = browser.find_by_text('Next 8 Hours')
        print('got next 8 hour spotted')
        next8.click()
        print('clicked and waiting')
        time.sleep(.5)
        print(f'end n={n} th pass')
    return

In [10]:
shutil.which('chromedriver')

'/Users/mastacow/data/forcast-forcast/env/bin/chromedriver'

In [11]:
browser = chrome()

In [12]:
browser.visit(url)

In [13]:
zip_and_click(code)

exception found....waiting...
no exception this time


In [14]:
now = scrape_now(browser)

In [18]:
next_36 = scrape_next36(browser)    

looking for now snapshot
completed section 0 snapshot
looking for at_start snapshot
completed section 1 snapshot
looking for at_12 snapshot
completed section 2 snapshot
looking for at_24 snapshot
ElementNotInteractableException occured: breaking loop.


In [20]:
goto_hourly(browser)

1
got next 8 hour spotted
clicked and waiting
end n=1 th pass
2
got next 8 hour spotted
clicked and waiting
end n=2 th pass
3
got next 8 hour spotted
clicked and waiting
end n=3 th pass
4
got next 8 hour spotted
clicked and waiting
end n=4 th pass
5
got next 8 hour spotted
clicked and waiting
end n=5 th pass


In [20]:
browser.quit()

In [55]:
now

['ADVANCE, NC',
 'as of 11:27 pm EST',
 '43°',
 'CLEAR',
 'feels like 41°',
 'H -- L 35°',
 'UV Index 0 of 10',
 'Holiday Week Travel Forecast',
 'RIGHT NOW',
 'Wind',
 'NNW 4 mph',
 'Humidity',
 '57%',
 'Dew Point',
 '29°',
 'Pressure',
 '30.31 in',
 'Visibility',
 '10.0 mi']

In [57]:
from pprint import pprint
pprint(next_36['now'])
# next_36

{'chance_precip': '0%',
 'condition': 'MOSTLY CLEAR',
 'description': 'Clear to partly cloudy. Low near 35F. Winds light and '
                'variable.',
 'high/low': 'LOW',
 'hour': 'CALCULATED HOUR OF PREDICTION',
 'humidity': '68%',
 'sun': {'rise': '7:10 am5:08 pm', 'set': '7:10 am5:08 pm'},
 'temp_f': '35°',
 'uv_index': '0 of 10',
 'wind': {'direction': 'NNE', 'speed': '4'}}


In [21]:
def get_table(browser):
    ''' Get the tables from the current page and turn to pandas df's '''
    url = browser.url
    r = requests.get(url)
    dfs = pd.read_html(r.text)
    return(dfs)

In [22]:
get_table(browser)

[    Time  Description        Temp Feels Precip Humidity Wind Unnamed: 7
 0    NaN  12:30 pmSat      Cloudy   48°    48°       0%  86%  SSE 2 mph
 1    NaN   1:00 pmSat      Cloudy   49°    49°       0%  85%  SSE 2 mph
 2    NaN   2:00 pmSat      Cloudy   50°    50°       0%  81%  SSE 2 mph
 3    NaN   3:00 pmSat      Cloudy   50°    50°      10%  78%  SSE 3 mph
 4    NaN   4:00 pmSat      Cloudy   50°    50°      25%  77%   SE 2 mph
 5    NaN   5:00 pmSat        Rain   50°    50°      70%  80%  ESE 1 mph
 6    NaN   6:00 pmSat     Showers   49°    49°      65%  89%  ESE 1 mph
 7    NaN   7:00 pmSat        Rain   48°    48°      60%  93%   SE 2 mph
 8    NaN   8:00 pmSat  Light Rain   47°    47°      65%  93%   SE 2 mph
 9    NaN   9:00 pmSat  Light Rain   47°    47°      65%  94%  ESE 2 mph
 10   NaN  10:00 pmSat        Rain   47°    47°      85%  96%  ESE 2 mph
 11   NaN  11:00 pmSat        Rain   47°    47°      90%  95%   SE 2 mph
 12   NaN  12:00 amSun        Rain   47°    46°    

In [36]:
browser.back()

In [35]:
len(dfs)

1

In [19]:
# A line that will click out of an advertisement
browser.find_by_tag('a#bx-close-inside-970188')[0].click()

In [ ]:
# snapshot = browser.find_by_tag('div#daypart-0').value
# snaplist = snapshot.split('\n')
# details = browser.find_by_tag('span.wx-detail-value')
# detailslist = [details[i].value for i in range(4)]
# print(snaplist)
# detailslist

In [49]:
print(snaplist)

['TONIGHT', 'RAIN', 'LOW', '42°', '90%']


None


NNW 10 mph
86%
0 of 10
6:43 am5:13 pm


'TODAY\nHIGH\n70°\n10%'

'A clear sky. Scattered frost possible. Low around 30F. Winds light and variable.'

In [40]:
browser.quit()

1

'Today'

got input box


1

In [25]:
hour_code = {'0':'now',
            '1':'at_start',
            '2':'at_12',
            '3':'at_24',
            '4':'at_36'
}
print(hour_code['0'])
for i in range(5):
    print(hour_code[str(i)])
    print(f'looking for {hour_code[str(i)]} snapshot')

now
now
looking for now snapshot
at_start
looking for at_start snapshot
at_12
looking for at_12 snapshot
at_24
looking for at_24 snapshot
at_36
looking for at_36 snapshot


In [ ]:
### SEARCH FOR THE TABLE THAT HOLDS THE SEARCH RESULTS ###